In [1]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from sklearn import preprocessing, tree
from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import KFold

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df=pd.read_json('../01_Preprocessing/First.json').sort_index()

In [3]:
df.head(2)

,country,countryCoded,degree,degreeCoded,eng,engCoded,fieldGroup,fund,gpaBachelors,gpaMasters,gre,highLevelBachUni,highLevelMasterUni,paper,targetUni,uniRank,year
2,united kingdom,4,Doctorate,1,High,1,7,TRUE,3.5,3.442986,FALSE,FALSE,FALSE,6.666667,heriot-watt,692.0,2007
3,united states,0,Doctorate,1,VeryHigh,0,5,TRUE,2.5,3.500000,TRUE,TRUE,FALSE,11.333333,northwestern,20.0,2010


In [4]:
def mydist(x, y):
    return np.sum((x-y)**2)
def jaccard(a, b):
    intersection = float(len(set(a) & set(b)))
    union = float(len(set(a) | set(b)))
    return 1.0 - (intersection/union)

In [5]:
# http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html
    
dist=['braycurtis','canberra','chebyshev','cityblock','correlation','cosine','euclidean','dice','hamming','jaccard','kulsinski','matching','rogerstanimoto','russellrao','sokalsneath','yule']
algorithm=['ball_tree', 'kd_tree', 'brute']

In [6]:
len(dist)

16

## On country (only MS)

In [7]:
df.fund= df.fund=='TRUE'
df.gre= df.gre=='TRUE'
df.highLevelBachUni= df.highLevelBachUni=='TRUE'
df.highLevelMasterUni= df.highLevelMasterUni=='TRUE'
df.uniRank.fillna(294,inplace=True)

In [8]:
df.columns

Index([u'country', u'countryCoded', u'degree', u'degreeCoded', u'eng',
       u'engCoded', u'fieldGroup', u'fund', u'gpaBachelors', u'gpaMasters',
       u'gre', u'highLevelBachUni', u'highLevelMasterUni', u'paper',
       u'targetUni', u'uniRank', u'year'],
      dtype='object')

In [9]:
oldDf=df.copy()
df=df[['countryCoded','degreeCoded','engCoded', 'fieldGroup','fund','gpaBachelors','gre', 'highLevelBachUni', 'paper','uniRank']]
df=df[df.degreeCoded==0]
del df['degreeCoded']
bestAvg=[]
for alg in algorithm:
    for dis in dist:
        k_fold = KFold(n=len(df), n_folds=5)
        scores = []
        try:
            clf = KNeighborsClassifier(n_neighbors=3, weights='distance',algorithm=alg, metric=dis)
        except Exception as err:
#             print(alg,dis,'err')
            continue
        for train_indices, test_indices in k_fold:
            xtr = df.iloc[train_indices,(df.columns != 'countryCoded')]
            ytr = df.iloc[train_indices]['countryCoded']
            xte = df.iloc[test_indices, (df.columns != 'countryCoded')]
            yte = df.iloc[test_indices]['countryCoded']
            clf.fit(xtr, ytr)
            ypred = clf.predict(xte)
            acc=accuracy_score(list(yte),list(ypred))
            scores.append(acc*100)
        print(alg,dis,np.average(scores))
        bestAvg.append(np.average(scores))
print('>>>>>>>Best: ',np.max(bestAvg))

('ball_tree', 'braycurtis', 55.507529507529512)
('ball_tree', 'canberra', 44.839072039072036)
('ball_tree', 'chebyshev', 53.738054538054541)
('ball_tree', 'cityblock', 55.735775335775337)
('ball_tree', 'euclidean', 55.793080993080991)
('ball_tree', 'dice', 46.14798534798534)
('ball_tree', 'hamming', 47.408547008547011)
('ball_tree', 'jaccard', 46.14798534798534)
('ball_tree', 'kulsinski', 46.319413919413918)
('ball_tree', 'matching', 46.14798534798534)
('ball_tree', 'rogerstanimoto', 46.14798534798534)
('ball_tree', 'russellrao', 48.896052096052095)
('ball_tree', 'sokalsneath', 46.14798534798534)
('kd_tree', 'chebyshev', 53.909483109483105)
('kd_tree', 'cityblock', 55.67863247863248)
('kd_tree', 'euclidean', 55.793080993080991)
('brute', 'braycurtis', 55.393080993081)
('brute', 'canberra', 45.066829466829468)
('brute', 'chebyshev', 53.738217338217339)
('brute', 'cityblock', 55.735449735449741)
('brute', 'correlation', 42.444444444444443)
('brute', 'cosine', 44.841025641025645)
('brute'

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, _DataConversionWarning)


('brute', 'dice', 41.181115181115182)
('brute', 'hamming', 46.378184778184774)
('brute', 'jaccard', 41.181115181115182)
('brute', 'kulsinski', 41.181115181115182)
('brute', 'matching', 41.181115181115182)
('brute', 'rogerstanimoto', 41.181115181115182)
('brute', 'russellrao', 50.433211233211232)
('brute', 'sokalsneath', 41.181115181115182)


C:\ProgramData\Anaconda2\lib\site-packages\sklearn\utils\extmath.py:483: RuntimeWarning: invalid value encountered in greater
  mostfrequent = np.where(counts > oldcounts, score, oldmostfreq)
C:\ProgramData\Anaconda2\lib\site-packages\sklearn\utils\extmath.py:484: RuntimeWarning: invalid value encountered in maximum
  oldcounts = np.maximum(counts, oldcounts)


('brute', 'yule', 49.464061864061861)
('>>>>>>>Best: ', 55.793080993080991)


## On Fund (only MS)

In [10]:
bestAvg=[]
for alg in algorithm:
    for dis in dist:
        k_fold = KFold(n=len(df), n_folds=5)
        scores = []
        try:
            clf = KNeighborsClassifier(n_neighbors=3, weights='distance',algorithm=alg, metric=dis)
        except Exception as err:
            continue
        for train_indices, test_indices in k_fold:
            xtr = df.iloc[train_indices, (df.columns != 'fund')]
            ytr = df.iloc[train_indices]['fund']
            xte = df.iloc[test_indices, (df.columns != 'fund')]
            yte = df.iloc[test_indices]['fund']
            clf.fit(xtr, ytr)
            ypred = clf.predict(xte)
            acc=accuracy_score(list(yte),list(ypred))
            score=acc*100
            scores.append(score)
            if (len(bestAvg)>1) :
                if(score > np.max(bestAvg)) :
                    bestClf=clf
        bestAvg.append(np.average(scores))
        print (alg,dis,np.average(scores))
print('>>>>>>>Best: ',np.max(bestAvg))

('ball_tree', 'braycurtis', 76.495400895400905)
('ball_tree', 'canberra', 75.354008954008961)
('ball_tree', 'chebyshev', 75.584533984533977)
('ball_tree', 'cityblock', 77.293935693935694)
('ball_tree', 'euclidean', 76.496703296703302)
('ball_tree', 'dice', 74.383557183557173)
('ball_tree', 'hamming', 76.152706552706562)
('ball_tree', 'jaccard', 74.383557183557173)
('ball_tree', 'kulsinski', 74.497842897842901)
('ball_tree', 'matching', 74.383557183557173)
('ball_tree', 'rogerstanimoto', 74.383557183557173)
('ball_tree', 'russellrao', 75.409361009360993)
('ball_tree', 'sokalsneath', 74.383557183557173)
('kd_tree', 'chebyshev', 75.641676841676855)
('kd_tree', 'cityblock', 77.293935693935694)
('kd_tree', 'euclidean', 76.553683353683354)
('brute', 'braycurtis', 76.495563695563703)
('brute', 'canberra', 75.411151811151825)
('brute', 'chebyshev', 75.754008954008967)
('brute', 'cityblock', 77.008547008547012)
('brute', 'correlation', 73.528367928367928)
('brute', 'cosine', 72.901912901912894)

### Best : ('kd_tree', 'cityblock', 77.692144892144896)

In [14]:
me=[0,2,0,2.5,False,False,1.5,400]

In [15]:
n=bestClf.kneighbors([me])
n

(array([[ 0.,  0.,  0.]]), array([[225, 255, 348]], dtype=int64))

In [16]:
for i in n[1]:
    print(xtr.iloc[i])

     countryCoded  engCoded  fieldGroup  gpaBachelors    gre highLevelBachUni  \
664             0         2           0           2.5  False            False   
767             0         2           0           3.0  False            False   
911             0         2           0           3.0  False            False   

        paper  uniRank  
664  1.000000     72.0  
767  1.333333    294.0  
911  3.000000    294.0  
